In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

In [10]:
chat = ChatOpenAI(
    temperature=0.2,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

In [3]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [4]:
example_template = '''
    Human: {question}
    AI: {answer}
'''

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Germany")

'\n    Human: What do you know about France?\n    AI: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\n\n\n    Human: What do you know about Italy?\n    AI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\n\n\n    Human: What do you know about Greece?\n    AI: \n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\n\nHuman: What do you know about Germany?'

In [5]:
chain = prompt | chat

chain.invoke(
    {
        "country": "Germany"
    }
)

AI: 
        Here is what I know:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro

AIMessageChunk(content='AI: \n        Here is what I know:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro')

In [13]:
examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}")
    ]
)

example_few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# 기존 프롬프트에 few-shot 프롬프트를 추가!
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. You can give a long answer"),
    example_few_shot_prompt,
    ("human", "What do you know about {country}?")
])

chain = final_prompt | chat

chain.invoke({"country": "Thailand"})

Thailand is a country located in Southeast Asia known for its rich culture, stunning beaches, and delicious cuisine. Here are some key facts about Thailand:

Capital: Bangkok
Language: Thai
Currency: Thai Baht
Famous for: Temples, tropical islands, vibrant street markets, and spicy cuisine
Popular tourist destinations: Bangkok, Chiang Mai, Phuket, Krabi, and Pattaya
Religion: Predominantly Buddhist
Climate: Tropical with a monsoon season from May to October
Wildlife: Thailand is home to diverse wildlife, including elephants, tigers, and monkeys
Cultural attractions: The Grand Palace, Wat Arun, and Ayutthaya Historical Park
Traditional Thai arts: Thai massage, Muay Thai (Thai boxing), and traditional dance forms like Khon and Ram Thai

AIMessageChunk(content='Thailand is a country located in Southeast Asia known for its rich culture, stunning beaches, and delicious cuisine. Here are some key facts about Thailand:\n\nCapital: Bangkok\nLanguage: Thai\nCurrency: Thai Baht\nFamous for: Temples, tropical islands, vibrant street markets, and spicy cuisine\nPopular tourist destinations: Bangkok, Chiang Mai, Phuket, Krabi, and Pattaya\nReligion: Predominantly Buddhist\nClimate: Tropical with a monsoon season from May to October\nWildlife: Thailand is home to diverse wildlife, including elephants, tigers, and monkeys\nCultural attractions: The Grand Palace, Wat Arun, and Ayutthaya Historical Park\nTraditional Thai arts: Thai massage, Muay Thai (Thai boxing), and traditional dance forms like Khon and Ram Thai')